In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data
from tqdm import tqdm

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import client_surveillance_jupyter
from typing import Union,cast


In [4]:
TRANSFER_PACKAGES_START_DATE = '2021-01-01'
SALES_TRANSACTIONS_START_DATE = '2021-01-01'

In [5]:
ca_df = pd.read_csv('retailer_nov2022.csv',index_col = 0)
ca_df.columns = ['company_name','company_identifier','state','licenses']
ca_df['company_identifier'] = ca_df['company_identifier'].astype(str)

In [6]:
ca_df_clean = ca_df[~ca_df['company_identifier'].isin(['PL','WHT','GF','UR'])]
ca_df_clean.shape

(94, 4)

In [7]:
ca_df_clean['lic_list'] = [list(ca_df_clean['licenses'])[i].split(";") for i in range(len(ca_df_clean))]

In [8]:
ca_df_clean = ca_df_clean[ca_df_clean['state'] == 'CA']

In [9]:
ca_df_clean['company_identifier'].nunique()

68

In [10]:
ca_df

,company_name,company_identifier,state,licenses
0,64 and Hope Melrose,64HM,CA,C10-0000922-LIC
1,99 High Tide,99HT,CA,C10-0000279-LIC
2,A Green Alternative Inc.,AGA,CA,C10-0000341-LIC
3,Altum LLC,ALT,MI,PC-000310;AU-R-000380
4,Bella Sol,BS,MI,PC-000185;AU-R-000156
...,...,...,...,...
91,The Good Leaf,TGL,CA,C9-0000412-LIC
92,The Joy Reserve,TJR,CA,C9-0000105-LIC
93,The Twisted Sista,TTS,MI,AU-R-000546;PC-000718
94,Three Trees,TT,CA,C9-0000451-LIC;C9-0000551-LIC


In [11]:
def get_identifiers_for_where_clause(company_identifier
):

    if type(company_identifier) == str:
        identifiers.append(cast(str, company_identifier))
    else:
        identifiers = cast(List[str], company_identifier)

    return ['"{}"'.format(iden) for iden in identifiers]

In [14]:
def create_query(LICENSE_NUMBER_LIST):
    return f"""select
  COALESCE(
    company_licenses.company_id, incoming.shipper_facility_license_number
  ) shipper_company_id,
  COALESCE(
    companies.parent_company_id, incoming.shipper_facility_license_number
  ) shipper_parent_company_id,
  COALESCE(
    parent_companies.name, incoming.shipper_facility_name
  ) shipper_parent_name,
  incoming.created_date,
  incoming.received_datetime,
  incoming.shipper_facility_name,
  incoming.recipient_facility_name,
  incoming.package_id,
  incoming.type,
  incoming.product_name,
  incoming.shipper_wholesale_price,
  incoming.shipped_quantity,
  incoming.receiver_wholesale_price,
  incoming.updated_at
from
  dbt_transformation.int__company_incoming_transfer_packages incoming
  left join dbt_transformation.stg__company_licenses company_licenses on incoming.shipper_facility_license_number = company_licenses.license_number
  left join dbt_transformation.stg__companies companies on company_licenses.company_id = companies.id
  left join dbt_transformation.stg__parent_companies parent_companies on companies.parent_company_id = parent_companies.id
WHERE
    incoming.recipient_facility_license_number IN ({','.join(LICENSE_NUMBER_LIST)})
    AND DATE(incoming.created_date) >= DATE('2021-01-01')
    AND DATE(incoming.created_date) < DATE('2022-11-01')
    
    """

In [15]:
def read_df_in(license_numbers):
    company_incoming_transfer_packages_query = create_query(
    LICENSE_NUMBER_LIST=license_numbers,
    )
    company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
    return company_incoming_transfer_packages_dataframe


In [16]:
WINDOW = 4
VC_START_DATE = '2021-01-01'
VC_END_DATE = '2022-10-01'
VC_MONTH_LIST = ['2022-07-01','2022-08-01','2022-09-01','2022-10-01']
VC_MONTH_END = VC_MONTH_LIST[-1]

In [17]:
def get_vc_list(company_list,transfer_packages_start_date,sales_transactrions_start_date,map_df):
    vc_df_list = {}
    for c in tqdm(company_list):
        vc_list = {}
        print(c)
        l_list = map_df[map_df['company_identifier'] == c]['lic_list'].to_list()[0]
        for l in tqdm(l_list):
            print(l)
            df_in = read_df_in(get_identifiers_for_where_clause([l]))
            if df_in.shape[0] == 0:
                vc_list[l] = numpy.nan
                continue
            else:
                vc = client_surveillance_jupyter.calculate_vendor_churn_short(df_in,[l],VC_START_DATE,VC_END_DATE,VC_MONTH_LIST,VC_MONTH_END)[1]
                vc_list[l] = vc
        vc_df_list[c] =  vc_list
    return vc_df_list


In [18]:
c = get_vc_list(list(ca_df_clean['company_identifier']),TRANSFER_PACKAGES_START_DATE,\
                SALES_TRANSACTIONS_START_DATE,ca_df_clean)

  0%|          | 0/68 [00:00<?, ?it/s]

64HM



  0%|          | 0/1 [00:00<?, ?it/s]

C10-0000922-LIC



  1%|▏         | 1/68 [00:04<04:54,  4.40s/it]

...


 99%|█████████▊| 67/68 [06:09<00:05,  5.72s/it]

T4L



  0%|          | 0/1 [00:00<?, ?it/s]

C12-0000302-LIC



100%|██████████| 68/68 [06:11<00:00,  5.47s/it]


In [21]:
df_vc = pd.DataFrame()
for k in c.keys():
    print(k)
    for l in c[k].keys():
        print(l)
        data_vc = c[k][l]
        if str(data_vc) == 'nan':
            continue
        else:
            data_vc['company'] = k
            data_vc['location'] = l
            print(data_vc.shape)
            df_vc = df_vc.append(data_vc)

64HM
C10-0000922-LIC
(84, 32)
99HT
C10-0000279-LIC
(143, 32)
AGA
C10-0000341-LIC
(92, 32)
BUD
C9-0000464-LIC
(33, 32)
C9-0000167-LIC
(39, 32)
C9-0000399-LIC
(49, 32)
C9-0000467-LIC
(27, 32)
C9-0000444-LIC
(27, 32)
BRC
C10-0000603-LIC
(154, 32)
BYN
C10-0000528-LIC
(89, 32)
CG
C9-0000157-LIC
(80, 32)
CPA
C9-0000348-LIC
(52, 32)
C10-0000461-LIC
(107, 32)
CPC
C9-0000056-LIC
(40, 32)
CSC
C10-0000670-LIC
(131, 32)
CSCC
C10-0000918-LIC
(60, 32)
CCC
C12-0000087-LIC
(102, 32)
CLC
C10-0000986-LIC
(19, 32)
C12-0000318-LIC
(16, 32)
CC
C10-0000758-LIC
(106, 32)
CED
C9-0000094-LIC
(32, 32)
CRRH
C12-0000308-LIC
(12, 32)
C12-0000405-LIC
(55, 32)
CE
C12-0000334-LIC
(44, 32)
DGG
C9-0000016-LIC
(53, 32)
EL
C9-0000370-LIC
(30, 32)
EMA
C10-0000774-LIC
(152, 32)
EMB
C10-0001096-LIC
(42, 32)
EMF
C10-0000939-LIC
(52, 32)
EMM
C10-0000786-LIC
(106, 32)
EMT
C10-0000695-LIC
(109, 32)
EPC
C10-0000034-LIC
(113, 32)
FI
C10-0001025-LIC
(37, 32)
C10-0000606-LIC
(159, 32)
C10-0000753-LIC
(41, 32)
C12-0000400-LIC
(37, 3

In [22]:
df_vc_state = df_vc.merge(ca_df_clean[['company_identifier','state']],how = 'left',left_on = 'company',right_on = 'company_identifier')

# tiering analysis

In [23]:
tier_a_vendor = set(['Herbl',
'Dreamfields',
'Kiva Sales & Service',
'Artisan Canna Cigars',
'NW Confections CA',
'Falcon',
'Irrational Raccoon',
'Grupo Flor (Vendor)',
'Claybourne Industries',
'Umbrla',
'Nabis',
'Uprooted (San Ysidro)',
'Ironworks Collective',
'1346 LA LLC',
'Greenfield Organix dba Loudpack',
'HGDCO Inc',
'Growpacker',
'PRUSSIAN STICKS, LLC',
'NorCal Distribution Solutions',
'MDS BUSINESS SERVICES LLC'])

In [24]:
tier_b_vendor = set(['Perfect Union',
'PRUSSIAN STICKS LLC',
'Mammoth Distro (Heavy Hitters)',
'UpNorth Humboldt',
'Standard Holdings',
'Herbarium',
'Big Tree Industries',
'Connected Cannabis Co',
'Four Star Manufacturing',
'Good Tree',
'E&J Distributors',
'River Distribution (Continuum)',
'VARAVO',
'Cypress Manufacturing Co',
'Coastal Distribution',
'Kind House',
'Melrose Facility',
'ASV EQ 1, LLC Dba 710 Labs',
'Aureus DHS',
'SCISLC, LLC'])

In [25]:
tier_c_vendor = set(['JBTB HOLDINGS INC',
'HERBAL REMEDIES CAREGIVERS, INC.',
'Connected Transportation Partners',
"EH Tech Inc. dba Nature's Market",
'WCC MGMT',
'Liquid Shade',
'Sunderstorm',
'Sublime Machining',
'JC Rad',
'Greenleaf'])

In [26]:
df_vc_state

,shipper_parent_name,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,facility_total,grand_total,perc_total,last_4m_total,last_4m_active,significant,measure,company,location,company_identifier,state
0,420-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2586.57,0.0,1264.33,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,3850.90,313958.48,0.012266,0.0,False,True,Inactive,64HM,C10-0000922-LIC,64HM,CA
1,"66 MAIN GROUP, LLC",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,313958.48,0.000000,0.0,False,False,Exclude,64HM,C10-0000922-LIC,64HM,CA
2,"ALYER DISTRIBUTION, INC",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,313958.48,0.000000,0.0,False,False,Exclude,64HM,C10-0000922-LIC,64HM,CA
3,ANTRINITY LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,313958.48,0.000000,0.0,False,False,Exclude,64HM,C10-0000922-LIC,64HM,CA
4,Alkhemist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,313958.48,0.000000,0.0,False,False,Exclude,64HM,C10-0000922-LIC,64HM,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,"Sundberg & Schackow Distribution, Inc.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,138093.51,0.000000,0.0,False,False,Exclude,T4L,C12-0000302-LIC,T4L,CA
7178,Tree4Life,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,138093.51,0.000000,0.0,False,False,Exclude,T4L,C12-0000302-LIC,T4L,CA
7179,VARAVO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,2031.95,0.0,0.0,0.0,0.0,0.0,2031.95,138093.51,0.014714,0.0,False,True,Inactive,T4L,C12-0000302-LIC,T4L,CA
7180,West Coast Sunrise Industries Cooperative Corp.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,138093.51,0.000000,0.0,False,False,Exclude,T4L,C12-0000302-LIC,T4L,CA


In [48]:
churn_a_list = {}
churn_b_list = {}
churn_c_list = {}
total_churn_list = {}
purchase_a_list = {}
purchase_b_list = {}
purchase_c_list = {}
total_purchase_list = {}

for c in df_vc_state['location'].unique():
    vc_c = df_vc_state[df_vc_state['location'] == c]
    i = vc_c['company_identifier'].unique()
    print(i)
    #churn
    total_churn = vc_c[vc_c['measure'] == 'Inactive']['perc_total'].sum()
    total_churn_list[c] = total_churn
    #purchase
    total_purchase = vc_c['last_4m_total'].sum()
    total_purchase_list[c] = total_purchase
    if len(set(vc_c['shipper_parent_name'].unique()).intersection(tier_a_vendor)) == 0:
        print(c)
    else:
        #churn
        churn_df_a = vc_c[(vc_c['shipper_parent_name'].isin(list(tier_a_vendor)))&\
                          (vc_c['measure'] == 'Inactive')]['perc_total'].sum()
        churn_df_b = vc_c[(vc_c['shipper_parent_name'].isin(list(tier_b_vendor)))&\
                          (vc_c['measure'] == 'Inactive')]['perc_total'].sum()
        churn_df_c = vc_c[(vc_c['shipper_parent_name'].isin(list(tier_c_vendor)))&\
                          (vc_c['measure'] == 'Inactive')]['perc_total'].sum()
        #purchase
        purchase_df_a = vc_c[vc_c['shipper_parent_name'].isin(list(tier_a_vendor))]['last_4m_total'].sum()
        purchase_df_b = vc_c[vc_c['shipper_parent_name'].isin(list(tier_b_vendor))]['last_4m_total'].sum()
        purchase_df_c = vc_c[vc_c['shipper_parent_name'].isin(list(tier_c_vendor))]['last_4m_total'].sum()
        
    churn_a_list[c] = churn_df_a
    churn_b_list[c] = churn_df_b
    churn_c_list[c] = churn_df_c
    purchase_a_list[c] = purchase_df_a
    purchase_b_list[c] = purchase_df_b
    purchase_c_list[c] = purchase_df_c

['64HM']
['99HT']
['AGA']
['BUD']
['BUD']
['BUD']
['BUD']
['BUD']
['BRC']
['BYN']
['CG']
['CPA']
['CPA']
['CPC']
['CSC']
['CSCC']
['CCC']
['CLC']
['CLC']
C12-0000318-LIC
['CC']
['CED']
['CRRH']
['CRRH']
['CE']
['DGG']
['EL']
['EMA']
['EMB']
['EMF']
['EMM']
['EMT']
['EPC']
['FI']
['FI']
['FI']
['FI']
['GB2']
['GD']
['GD']
['GTC']
['GT']
C9-0000385-LIC
['GT']
['GL']
['GRG']
['GRG']
['GHC']
['GFEE']
['GFWF']
['HB']
['HB']
['HB']
['HS']
['HGD']
['HGD']
['HPCC']
['HPCC']
['HC']
['IDC']
['ID']
['LBC']
['MSS']
['ML']
['MW']
['MPW']
['MPW']
['MD']
['MR']
['ND']
['NCDS']
['NCDS']
['PRP']
['PRP']
['PRP']
['PRL']
['PU']
['PU']
['PU']
['PU']
['PU']
['PU']
['PU']
['PU']
['PU']
['PU']
['PU']
['PU']
['PU']
['PEH']
C10-0000827-LIC
['RA']
['RGD']
['SGF']
['SFV']
['SV']
['SLCC']
['SO']
['SO']
['SO']
C12-0000328-LIC
['ST']
['FD']
['TGL']
['TJR']
['TT']
['TT']
['T4L']


In [50]:
purchase_a_list

{'C10-0000922-LIC': 24531.370000000003,
 'C10-0000279-LIC': 69319.47000000002,
 'C10-0000341-LIC': 389099.44,
 'C9-0000464-LIC': 30353.7,
 'C9-0000167-LIC': 0.0,
 'C9-0000399-LIC': 20563.64,
 'C9-0000467-LIC': 4841.65,
 'C9-0000444-LIC': 0.0,
 'C10-0000603-LIC': 84441.16,
 'C10-0000528-LIC': 53382.399999999994,
 'C9-0000157-LIC': 3990.85,
 'C9-0000348-LIC': 137122.52999999997,
 'C10-0000461-LIC': 4761.4,
 'C9-0000056-LIC': 0.0,
 'C10-0000670-LIC': 143945.40999999997,
 'C10-0000918-LIC': 102241.31999999998,
 'C12-0000087-LIC': 413136.4,
 'C10-0000986-LIC': 6365.68,
 'C12-0000318-LIC': 6365.68,
 'C10-0000758-LIC': 21634.14,
 'C9-0000094-LIC': 50700.0,
 'C12-0000308-LIC': 0.0,
 'C12-0000405-LIC': 401829.35,
 'C12-0000334-LIC': 1113.0,
 'C9-0000016-LIC': 2317.52,
 'C9-0000370-LIC': 18940.0,
 'C10-0000774-LIC': 300519.79,
 'C10-0001096-LIC': 1515991.83,
 'C10-0000939-LIC': 285934.7,
 'C10-0000786-LIC': 300008.16,
 'C10-0000695-LIC': 287119.9,
 'C10-0000034-LIC': 50180.25,
 'C10-0001025-LIC'

In [ ]:
#C12-0000318-LIC : CLC does not look like it has valid VC data
#C9-0000385-LIC: GT
#C10-0000827-LIC: PEH
#C12-0000328-LIC: SO

In [56]:
total_churn_df = pd.DataFrame.from_dict(total_churn_list,orient ='index').reset_index()
churn_a_df = pd.DataFrame.from_dict(churn_a_list,orient ='index').reset_index()
churn_b_df = pd.DataFrame.from_dict(churn_b_list,orient ='index').reset_index()
churn_c_df = pd.DataFrame.from_dict(churn_c_list,orient ='index').reset_index()

In [57]:
total_purchase_df = pd.DataFrame.from_dict(total_purchase_list,orient ='index').reset_index()
purchase_a_df = pd.DataFrame.from_dict(purchase_a_list,orient ='index').reset_index()
purchase_b_df = pd.DataFrame.from_dict(purchase_b_list,orient ='index').reset_index()
purchase_c_df = pd.DataFrame.from_dict(purchase_c_list,orient ='index').reset_index()

In [58]:
total_purchase_df

,index,0
0,C10-0000922-LIC,64975.88
1,C10-0000279-LIC,304971.07
2,C10-0000341-LIC,1110230.32
3,C9-0000464-LIC,61965.29
4,C9-0000167-LIC,0.00
...,...,...
99,C9-0000412-LIC,24906.85
100,C9-0000105-LIC,81698.57
101,C9-0000451-LIC,50625.00
102,C9-0000551-LIC,817661.75


In [29]:
vc_backtest_df = total_churn_df.merge(churn_a_df,how = 'left',on = 'index')\
.merge(churn_b_df,how = 'left',on = 'index')\
.merge(churn_c_df,how = 'left',on = 'index')

In [59]:
vc_backtest_df_purchase = total_purchase_df.merge(purchase_a_df,how = 'left',on = 'index')\
.merge(purchase_b_df,how = 'left',on = 'index')\
.merge(purchase_c_df,how = 'left',on = 'index')

In [62]:
vc_backtest_df_purchase

,license,total,tier_a,tier_b,tier_c
0,C10-0000922-LIC,64975.88,24531.37,3557.14,4974.91
1,C10-0000279-LIC,304971.07,69319.47,24426.78,42362.91
2,C10-0000341-LIC,1110230.32,389099.44,162294.28,17280.63
3,C9-0000464-LIC,61965.29,30353.70,31611.59,0.00
4,C9-0000167-LIC,0.00,0.00,0.00,0.00
...,...,...,...,...,...
99,C9-0000412-LIC,24906.85,3112.40,6329.63,1464.99
100,C9-0000105-LIC,81698.57,9973.86,1224.18,7075.60
101,C9-0000451-LIC,50625.00,50625.00,0.00,0.00
102,C9-0000551-LIC,817661.75,243860.61,69510.07,110036.47


In [30]:
vc_backtest_df.columns = ['license','total','tier_a','tier_b','tier_c']


In [61]:
vc_backtest_df_purchase.columns = ['license','total','tier_a','tier_b','tier_c']


In [32]:
vc_backtest_df = vc_backtest_df.merge(ca_df_clean[['licenses','company_identifier']],\
                                      how = 'left',left_on = 'license',right_on = 'licenses')

In [33]:
vc_backtest_df

,license,total,tier_a,tier_b,tier_c,licenses,company_identifier
0,C10-0000922-LIC,0.269650,0.002487,0.047037,0.006880,C10-0000922-LIC,64HM
1,C10-0000279-LIC,0.167851,0.042817,0.004319,0.000000,C10-0000279-LIC,99HT
2,C10-0000341-LIC,0.089070,0.000000,0.000000,0.010896,C10-0000341-LIC,AGA
3,C9-0000464-LIC,0.778025,0.301562,0.038728,0.023610,NaN,NaN
4,C9-0000167-LIC,1.000000,0.000000,0.000000,1.000000,NaN,NaN
...,...,...,...,...,...,...,...
99,C9-0000412-LIC,0.405991,0.089277,0.034001,0.000000,C9-0000412-LIC,TGL
100,C9-0000105-LIC,0.972233,0.961923,0.000000,0.001115,C9-0000105-LIC,TJR
101,C9-0000451-LIC,0.800437,0.175467,0.151559,0.106929,NaN,NaN
102,C9-0000551-LIC,0.031657,0.000000,0.021062,0.000000,NaN,NaN


In [63]:
#vc_backtest_df_purchase.to_csv('vc_backtest_purchase.csv')

In [41]:
df_vc_state[(df_vc_state['company_identifier'] == '99HT')].to_csv('99ht.csv')

In [42]:
ht = df_vc_state[(df_vc_state['company_identifier'] == '99HT')]
churn_df_a = ht[(ht['shipper_parent_name'].isin(list(tier_a_vendor)))&\
                          (ht['measure'] == 'Inactive')]

In [54]:
ht[(ht['shipper_parent_name'].isin(list(tier_a_vendor)))]

,shipper_parent_name,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,facility_total,grand_total,perc_total,last_4m_total,last_4m_active,significant,measure,company,location,company_identifier,state
84,1346 LA LLC,7275.61,4200.10,0.00,5160.53,5335.33,3888.16,4500.00,4563.74,13937.14,0.0,0.00,0.00,9744.21,0.00,0.00,6569.24,0.00,0.00,5952.38,0.00,0.00,7664.18,29930.01,1031538.75,0.029015,13616.56,True,True,Active,99HT,C10-0000279-LIC,99HT,CA
108,Claybourne Industries,8070.00,8737.00,0.00,0.00,6485.00,0.00,6648.00,0.00,0.00,0.0,6738.00,6338.16,0.00,0.00,0.00,0.00,7738.00,0.00,0.00,0.00,9796.45,0.00,30610.61,1031538.75,0.029675,9796.45,True,True,Active,99HT,C10-0000279-LIC,99HT,CA
117,Dreamfields,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4325.50,2251.25,6576.75,1031538.75,0.006376,6576.75,True,True,Active,99HT,C10-0000279-LIC,99HT,CA
126,Falcon,0.00,8280.00,0.00,12052.00,0.00,7596.00,7080.00,0.00,0.00,0.0,9282.19,0.00,2400.00,0.00,0.00,0.00,0.00,3000.24,0.00,0.00,0.00,0.00,14682.43,1031538.75,0.014234,0.00,False,True,Inactive,99HT,C10-0000279-LIC,99HT,CA
139,Greenfield Organix dba Loudpack,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,3120.72,0.00,0.00,6172.03,0.00,0.00,0.00,0.00,9292.75,1031538.75,0.009009,0.00,False,True,Inactive,99HT,C10-0000279-LIC,99HT,CA
142,Growpacker,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4500.50,0.00,4500.50,1031538.75,0.004363,4500.50,True,True,Active,99HT,C10-0000279-LIC,99HT,CA
146,HGDCO Inc,0.00,11920.04,13040.07,23615.04,10668.04,9159.09,8918.02,10602.02,10468.10,0.0,8968.02,0.00,8720.05,7798.02,7772.01,9431.06,10051.08,0.00,9336.02,9678.04,9134.42,0.00,80888.72,1031538.75,0.078416,28148.48,True,True,Active,99HT,C10-0000279-LIC,99HT,CA
152,Herbl,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3820.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,2800.00,0.00,5964.14,0.00,4480.24,0.00,0.00,13244.38,1031538.75,0.012839,4480.24,True,True,Active,99HT,C10-0000279-LIC,99HT,CA
154,Ironworks Collective,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,2925.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2925.00,1031538.75,0.002836,0.00,False,True,Inactive,99HT,C10-0000279-LIC,99HT,CA
155,Irrational Raccoon,2880.09,6860.11,27591.93,6540.13,0.00,13467.00,3224.03,0.00,0.00,0.0,0.00,0.00,0.00,1920.03,0.00,0.00,3840.24,0.00,0.00,0.00,0.00,0.00,5760.27,1031538.75,0.005584,0.00,False,True,Inactive,99HT,C10-0000279-LIC,99HT,CA
